### Objectives: 
#### To understand and explore the potential Information that can be extracted from this dataset 

In [1]:
import pandas as pd
#library to connect to red shift 
import psycopg2 
import os 

#to supress any warnings 
import warnings
warnings.filterwarnings('ignore')


In [2]:
#to import passwords and access information from .env file 

from dotenv import load_dotenv
load_dotenv()

dbname = os.getenv('dbname')
host = os.getenv('host')
port = os.getenv('port')
user = os.getenv('user')
password = os.getenv('password')



In [3]:
# connect to redshift
def connect_to_redshift (dbname, host, port, user, password): 
    connect = psycopg2.connect(dbname=dbname, host=host, port=port, user=user, password=password)
    print('connection to redshift made') 
    return connect
    

In [4]:
connect = connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [5]:
query = ''' select * 
            from bootcamp.online_transactions_cleaned'''

customer_data = pd.read_sql(query, connect)           

In [6]:
customer_data.head()

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536370,21791,VINTAGE HEADS AND TAILS CARD GAME,1.25,24,30.00,2010-12-01 08:45:00,u12583,France
1,536385,22783,SET 3 WICKER OVAL BASKETS W LIDS,19.95,1,19.95,2010-12-01 09:56:00,u1742,United Kingdom
2,536392,22128,PARTY CONES CANDY ASSORTED,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
3,536404,22469,HEART OF WICKER SMALL,1.65,12,19.80,2010-12-01 11:29:00,u16218,United Kingdom
4,536412,22274,FELTCRAFT DOLL EMILY,2.95,2,5.90,2010-12-01 11:49:00,u1792,United Kingdom


### EDA of customers dataset 

In [17]:
#number of rows and columns 
customer_data.shape

(399841, 9)

In [19]:
customer_data.info()
#no null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399841 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            399841 non-null  object        
 1   stock_code         399841 non-null  object        
 2   description        399841 non-null  object        
 3   price              399841 non-null  float64       
 4   quantity           399841 non-null  int64         
 5   total_order_value  399841 non-null  float64       
 6   invoice_date       399841 non-null  datetime64[ns]
 7   customer_id        399841 non-null  object        
 8   country            399841 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 27.5+ MB


In [20]:
#number of unique values present in each column 
customer_data.nunique()

invoice              21791
stock_code            3679
description           3644
price                  392
quantity               432
total_order_value     3480
invoice_date         20138
customer_id           4363
country                 37
dtype: int64

~400K rows for 4363 customers buying 21791 purchases. 


In [23]:
# by sorting invoices, it is clear that each row represents one item in the invoice, it's price and how many of it was orderd 
# there are negative values that represent cancelled orders
customer_data.sort_values(by=['invoice'], ascending=True)


,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
8416,536365,84406B,CREAM CUPID HEARTS COAT HANGER,2.75,8,22.00,2010-12-01 08:26:00,u1785,United Kingdom
8374,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34,2010-12-01 08:26:00,u1785,United Kingdom
10147,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,2010-12-01 08:26:00,u1785,United Kingdom
8577,536365,22752,SET 7 BABUSHKA NESTING BOXES,7.65,2,15.30,2010-12-01 08:26:00,u1785,United Kingdom
7150,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,2010-12-01 08:26:00,u1785,United Kingdom
...,...,...,...,...,...,...,...,...,...
352729,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,1.95,-12,-23.40,2011-12-09 09:57:00,u14397,United Kingdom
329070,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,-11,-9.13,2011-12-09 09:57:00,u14397,United Kingdom
339287,C581568,21258,VICTORIAN SEWING BOX LARGE,10.95,-5,-54.75,2011-12-09 11:57:00,u15311,United Kingdom
301048,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,1.25,-1,-1.25,2011-12-09 11:58:00,u17315,United Kingdom


In [27]:
# Is cancelled or refund order have the same invoice without C?
customer_data.loc[customer_data['invoice'] == 'C581490'] 

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
329070,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,-11,-9.13,2011-12-09 09:57:00,u14397,United Kingdom
352729,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,1.95,-12,-23.40,2011-12-09 09:57:00,u14397,United Kingdom


In [28]:
customer_data.loc[customer_data['invoice'] == '581490'] 

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country


In [30]:
#may be we can alocate it based on stock code, quantity and customer id?
customer_data.loc[(customer_data['stock_code'] == '23144') & (customer_data['customer_id'] == 'u14397')] 

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
89418,552575,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-05-10 12:25:00,u14397,United Kingdom
105538,563982,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-08-22 11:17:00,u14397,United Kingdom
109857,559883,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-07-13 11:34:00,u14397,United Kingdom
112426,562442,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-08-04 18:31:00,u14397,United Kingdom
218698,561067,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-07-24 14:26:00,u14397,United Kingdom
231091,565792,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,24,19.92,2011-09-07 09:37:00,u14397,United Kingdom
246456,567531,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,24,19.92,2011-09-21 09:37:00,u14397,United Kingdom
263452,571225,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,12,9.96,2011-10-14 13:41:00,u14397,United Kingdom
271517,575684,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.72,144,103.68,2011-11-10 15:57:00,u14397,United Kingdom
273506,573294,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,36,29.88,2011-10-28 15:32:00,u14397,United Kingdom


There is no clear pattern of which refuned/ retours belongs to which purchases. 

In [32]:
#group by: same invoice, same customer_id
customer_data.groupby('invoice').count()

,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
invoice,,,,,,,,
536365,7,7,7,7,7,7,7,7
536366,2,2,2,2,2,2,2,2
536367,12,12,12,12,12,12,12,12
536368,4,4,4,4,4,4,4,4
536369,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
C581470,1,1,1,1,1,1,1,1
C581484,1,1,1,1,1,1,1,1
C581490,2,2,2,2,2,2,2,2


In [33]:
customer_data.groupby(['customer_id','invoice']).count()

stock_code  description  price  quantity  \
customer_id invoice                                             
u12346      541431            1            1      1         1   
            C541433           1            1      1         1   
u12347      537626           31           31     31        31   
            542237           29           29     29        29   
            549222           24           24     24        24   
...                         ...          ...    ...       ...   
u18283      579673           50           50     50        50   
            580872           50           50     50        50   
u18287      554065           29           29     29        29   
            570715           38           38     38        38   
            573167            3            3      3         3   

                     total_order_value  invoice_date  country  
customer_id invoice                                            
u12346      541431                   1             1        1  
            C541433                  1             1        1  
u12347      537626                  31            31       31  
            542237                  29            29       29  
            549222                  24            24       24  
...                                ...           ...      ...  
u18283      579673                  50            50       50  
            580872                  50            50       50  
u18287      554065                  29            29       29  
            570715                  38            38       38  
            573167                   3             3        3  

[21791 rows x 7 columns]

Bad pipe message: %s [b'QU+oNPc2xjjT5sUbtALw==\r\nConnection: Upgrade\r\nUpgrade: websocket\r\nSec-WebSocke']
Bad pipe message: %s [b'xIXn4kapCZnmFX9rtyYw==\r\nConnection: Upgrade\r\nUpg']


### Business Objective: Bill penetration in Retail 
The goal is to give a report helps supply chain to determine: 
-  which items are most frequently ordered
-   large amounts of order
-    categorization of items based on description  

In [11]:
grouped_invoice = customer_data.groupby('invoice').count()

In [12]:
grouped_invoice

,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
invoice,,,,,,,,
536365,7,7,7,7,7,7,7,7
536366,2,2,2,2,2,2,2,2
536367,12,12,12,12,12,12,12,12
536368,4,4,4,4,4,4,4,4
536369,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...
C581470,1,1,1,1,1,1,1,1
C581484,1,1,1,1,1,1,1,1
C581490,2,2,2,2,2,2,2,2


In [16]:
customer_data.sort_values(by=['invoice'])

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
10147,536365,85123A,CREAM HANGING HEART T-LIGHT HOLDER,2.55,6,15.30,2010-12-01 08:26:00,u1785,United Kingdom
6912,536365,71053,WHITE MOROCCAN METAL LANTERN,3.39,6,20.34,2010-12-01 08:26:00,u1785,United Kingdom
10181,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,4.25,6,25.50,2010-12-01 08:26:00,u1785,United Kingdom
7150,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,3.39,6,20.34,2010-12-01 08:26:00,u1785,United Kingdom
8374,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,3.39,6,20.34,2010-12-01 08:26:00,u1785,United Kingdom
...,...,...,...,...,...,...,...,...,...
354912,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,1.95,-12,-23.40,2011-12-09 09:57:00,u14397,United Kingdom
327087,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,0.83,-11,-9.13,2011-12-09 09:57:00,u14397,United Kingdom
337433,C581568,21258,VICTORIAN SEWING BOX LARGE,10.95,-5,-54.75,2011-12-09 11:57:00,u15311,United Kingdom
327106,C581569,20979,36 PENCILS TUBE RED RETROSPOT,1.25,-5,-6.25,2011-12-09 11:58:00,u17315,United Kingdom


#### Task: find top 10 most frequent and ordered by large quantities in the invoices

In [25]:
#most repeated item
top_10_items = customer_data['stock_code'].value_counts().nlargest(10)

In [26]:
top_10_items

stock_code
85123A    2065
22423     1894
85099B    1659
47566     1409
84879     1405
20725     1346
22720     1224
22197     1110
23203     1107
20727     1099
Name: count, dtype: int64

In [39]:
#largest qauntities
df_big_quantities = customer_data[['stock_code','description', 'quantity']]

In [40]:
df_big_quantities.groupby(['stock_code', 'description']).sum().sort_values(by='quantity', ascending=False ).head(10)

,,quantity
stock_code,description,
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,53119
22197,POPCORN HOLDER,48689
85099B,JUMBO BAG RED RETROSPOT,44963
84879,ASSORTED COLOUR BIRD ORNAMENT,35215
85123A,CREAM HANGING HEART T-LIGHT HOLDER,34185
21212,PACK OF 72 RETROSPOT CAKE CASES,33386
23084,RABBIT NIGHT LIGHT,27045
22492,MINI PAINT SET VINTAGE,25880
22616,PACK OF 12 LONDON TISSUES,25305


In [42]:
df_frequently_ordered = customer_data[['stock_code','description', 'invoice']]

In [45]:
#frequently_ordered - should we use count or sum of invoices?
df_frequently_ordered.groupby(['stock_code', 'description']).count().sort_values(by='invoice', ascending=False).head(10)

,,invoice
stock_code,description,
85123A,CREAM HANGING HEART T-LIGHT HOLDER,2065
22423,REGENCY CAKESTAND 3 TIER,1894
85099B,JUMBO BAG RED RETROSPOT,1659
47566,PARTY BUNTING,1409
84879,ASSORTED COLOUR BIRD ORNAMENT,1405
20725,LUNCH BAG RED RETROSPOT,1346
22720,SET OF 3 CAKE TINS PANTRY DESIGN,1224
22197,POPCORN HOLDER,1110
23203,JUMBO BAG VINTAGE DOILEY,1107


In [ ]:
#data frames for most frequent and largest quantities

#### Task: Text Analysis, finding patterns in description?

In [49]:
# how many descriptions in Dataset

description_list= customer_data['description'].unique()

In [50]:
description_list

array(['VINTAGE HEADS AND TAILS CARD GAME ',
       'SET 3 WICKER OVAL BASKETS W LIDS', 'PARTY CONES CANDY ASSORTED',
       ..., 'BLACK VINT ART DEC CRYSTAL BRACELET',
       'PAPER CRAFT , LITTLE BIRDIE', 'LETTER "Z" BLING KEY RING'],
      dtype=object)

In [55]:
#counts of each word in description
counts = pd.Series(''.join(df_frequently_ordered.description).split()).value_counts()
counts

OF                    41209
BAG                   31258
HEART                 22915
VINTAGE               20221
RED                   18893
                      ...  
23WORLD                   1
HOLDERUnknownSET          1
FLOWERLARGE               1
CUPSSTRIPES               1
TOADSTOOLSROTATING        1
Name: count, Length: 79915, dtype: int64

#### Future Objectives: Classification of description, is there a possibily of supervised machine learning ?